In [ ]:
import tensorflow as tf 
import tensorflow_probability as tfp
import numpy as np 
import scipy
from scipy.ndimage import affine_transform
import matplotlib.pyplot as plt


In [ ]:
# plt.imshow(x_train[3])
# plt.show()


meshgrid_x,meshgrid_y=np.mgrid[:28,:28]


def  basemoment(image):
  mass=np.sum(image)
  mom_x=np.sum(image*meshgrid_x)/mass
  mom_y=np.sum(image*meshgrid_y)/mass
  return (mass,mom_x,mom_y)

def moments(image,i,j):
  com=basemoment(image)
  com_x=meshgrid_x-com[1]
  com_y=meshgrid_y-com[2]
  intense_x=np.sum((com_x**i)*(com_y**j)*image)/com[0]
  # intense_y=np.sum((com_y**j)*image)/com[0]
  return intense_x

def cov(image):
  arr=[[moments(image,2,0),moments(image,1,1)],
       [moments(image,1,1),moments(image,0,2)]]
  return arr

def rot_matrix(image):
  list_cov=cov(image)
  # var=list_cov[0][0]-list_cov[1][1]
  # varn=2*list_cov[0][1]
  # theta=0.5*np.arctan(varn/var)
  # var=np.var(image)
  corr=moments(image,1,1)/moments(image,2,0)
  rot_matrix=[[1,0],
             [corr,1]]

  # rot_matrix=[[np.cos(theta),-np.sin(theta)],
  #             [-np.sin(theta),np.cos(theta)]]
  return rot_matrix

def transform(data):
  buff=np.zeros(data.shape)
  for i in range(len(buff)):
    rot_mat=rot_matrix(data[i])
    centroid=basemoment(data[i])
    offset=[centroid[1],centroid[2]]-np.dot(rot_mat,[14,14])
    rot_mat=affine_transform(data[i],rot_mat,offset=offset)
    buff[i]=rot_mat
  
  return tf.cast(buff,dtype=tf.float32)




In [ ]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()
x_train=transform(x_train)
x_test=transform(x_test)



In [ ]:
def ProbitRegression():
    input=tf.keras.layers.Flatten(input_shape=(28,28))
    output=tf.keras.layers.Dense(10,activation=tfp.bijectors.NormalCDF())
    layers=[input,output]
    model=tf.keras.Sequential(layers)
    model.compile(loss='SparseCategoricalCrossentropy',optimizer='adam',metrics='accuracy')
    return model


In [ ]:
callback=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',mode='max',patience=10,restore_best_weights=1)
model=ProbitRegression()
model.fit(x_train/255,y_train,epochs=200,validation_data=(x_test/255,y_test),callbacks=callback)
